In [ ]:
cd '/home/schlecker/repos/planeteScripts'

In [ ]:
%load_ext autoreload
%autoreload 2
from make_list import *
from planeteOutput import *
import matplotlib.pyplot as plt

In [ ]:
from planetePlots import *
import utils

In [ ]:
# load an example population
popfile = '/home/schlecker/phd/planete/outputs/bernNov17/popu/popu.hdf5'
j35file = '/home/schlecker/phd/planete/outputs/J35/pop04f_J35.hd5'
pop = read_popHdf5(j35file, hierarchical=False, nSample=5)

In [ ]:
# load single simulation with 70 embryos
T1simFile = '/home/schlecker/phd/planete/outputs/singleruns/01Trappist1/'
T1sim = read_simFromFolder(T1simFile)

In [ ]:
T1sim['planet013'].status.tail(1) == -1

In [ ]:
truths = utils.get_Trappist1data()
fig, ax = plot_planetTracks(T1sim, truths)
# fig.savefig('/home/schlecker/Desktop/T1.svg', rasterized=True)

In [ ]:
# ax.scatter([7e6 for i in range(len(truths))], truths['a'])
plt.scatter([x*9e5 for x in range(8)], [x*.4 for x in range(8)], s=40, edgecolors='white', linewidth=1.5, zorder=10)
# plt.scatter([x*8e5 for x in range(8)], [x*.5 for x in range(8)], s=40, zorder=10)